# 0. train_raw 파일에 있는 jpg형식만 리스트에 저장

In [1]:
import os

path = "./train_raw"
file_list = os.listdir(path)
file_list_jpeg = [file for file in file_list if file.endswith(".jpeg")]

print ("file_list_jpeg: {}".format(file_list_jpeg))

file_list_jpeg: ['suho.jpeg', 'jiyun.jpeg', 'yearang.jpeg', 'seonghee.jpeg', 'jaewon.jpeg', 'hangyeol.jpeg']


In [2]:
file_list_jpeg = ['suho.jpeg']

# 1. 얼굴 부분만 데이터로 저장하기
## train_raw 파일 ==> train_cascade 파일

In [3]:
import cv2
import numpy as np

 
for j in range(len(file_list_jpeg)):
    # 테스트 이미지 불러오기
    img = cv2.imread('train_raw/'+file_list_jpeg[j])
    img = cv2.resize(img,(800,800))

    # RGB -> Gray로 변환
    # 얼굴 찾기 위해 그레이스케일로 학습되어 있기때문에 맞춰줘야 한다.
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)



    # 정면 얼굴 인식용 cascade xml 불러오기
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

    # 이미지내에서 얼굴 검출
    faces = face_cascade.detectMultiScale(img_gray, 1.3, 5)


    # 얼굴 검출되었다면 좌표 정보를 리턴 받고, 없으면 오류
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) # 원본 영상에 위치 표시
        roi_gray = img_gray[y:y+h, x:x+w] # roi 생성
        roi_color = img[y:y+h, x:x+w] # roi 

#     cv2.imshow('img',roi_color)
    save_to_dir='train_cascade/{}.jpeg'.format(file_list_jpeg[j][:-5])
    save_to_dir = "{}".format(save_to_dir)
    print(save_to_dir)

    cv2.imwrite(save_to_dir,roi_color)

    cv2.waitKey(0)
    cv2.destroyAllWindows()### 얼굴 부분만 데이터로 저장하기

train_cascade/suho.jpeg


# 2. Image Augmentation
## train_cascade 파일 ==> train_aug 파일

In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=20,        # 랜덤한 각도로 돌리기 (지정한 숫자 안에서)
        width_shift_range=0.1,    # 좌/우 이동
        height_shift_range=0.1,   # 위 아래 이동
        shear_range=0.9,          # 도 단위의 반시계 방향 층밀리기 각도
        zoom_range=[0.8,0.99],    # 랜덤하게 zoom
        brightness_range = [0.1,1.0],  # 랜덤한 이미지 밝기 (0:검정)
        horizontal_flip=True)     # 좌우 반전


for j in range(len(file_list_jpeg)):
    img = load_img('train_cascade/'+file_list_jpeg[j])
    x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
    x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir='train_aug/{}'.format(file_list_jpeg[j][:-5]), save_prefix='img {}'.format(file_list_jpeg[j]), save_format='jpeg'):
        i += 1
        if i > 500:
            break 

Using TensorFlow backend.
